# Minesweeper Puzzle

Lucerne University of Applied Sciences and Arts - School of Information Technology

The player is presented with a grid of squares. Some randomly selected squares, unknown to the player, contain mines. Other squares may display numbers indicating the count of mines present in the immediate neighborhood of the squares. Find all mines. 

See https://de.wikipedia.org/wiki/Minesweeper or go to http://minesweeperonline.com/ to play

Imports

In [1]:
from ortools.sat.python import cp_model
from itertools import product
import numpy as np

Some Minesweeper examples, the first one is from the lecture

In [2]:
mine_field1 = [
    ["2", "", "", "", "1"],
    ["2", "", "4", "3", ""],
    ["", "2", "", "1", ""],
    ["", "1", "", "3", ""],
    ["1", "", "", "", ""],
]
mine_field2 = [
    ["1", "", "", "", "2", "", "", "1"],
    ["", "1", "2", "", "3", "", "", "1"],
    ["", "2", "", "1", "", "", "", "0"],
    ["", "2", "1", "", "", "2", "3", "1"],
    ["", "", "", "2", "", "", "", ""],
    ["1", "", "", "", "4", "3", "", ""],
    ["", "1", "", "", "4", "", "3", ""],
    ["1", "", "", "2", "", "2", "", "1"],
]
mine_field3 = [
    ["", "", "", "", "", "1", "", ""],
    ["1", "2", "2", "1", "", "", "1", "1"],
    ["", "1", "", "", "", "", "", ""],
    ["", "", "0", "", "", "1", "", "1"],
    ["", "1", "", "", "", "", "", ""],
    ["", "", "1", "", "2", "3", "", ""],
    ["", "", "", "", "", "", "2", "1"],
    ["", "1", "", "2", "2", "", "2", ""],
]
mine_field4 = [
    ["1", "", "", "2", "", "2", "", "2", "", ""],
    ["", "3", "2", "", "", "", "4", "", "", "1"],
    ["", "", "", "1", "3", "", "", "", "4", ""],
    ["3", "", "1", "", "", "", "3", "", "", ""],
    ["", "2", "1", "", "1", "", "", "3", "", "2"],
    ["", "3", "", "2", "", "", "2", "", "1", ""],
    ["2", "", "", "3", "2", "", "", "2", "", ""],
    ["", "3", "", "", "", "3", "2", "", "", "3"],
    ["", "", "3", "", "3", "3", "", "", "", ""],
    ["", "2", "", "2", "", "", "", "", "2", ""],
]

Pick one of the examples

In [3]:
game = mine_field1

Create model

In [4]:
model = cp_model.CpModel()

In [5]:
# calculate neighbour-cells
def neighbours(cell_y, cell_x):
    neighbours = []
    for y in range(cell_y-1, cell_y+2):
        for x in range(cell_x-1, cell_x+2):
            if not (y < 0 or y == len(game)) and not (x < 0 or x == len(game)) and not(cell_y == y and cell_x == x):
                neighbours.append((y,x))
    return neighbours

In [6]:
board = [[model.NewBoolVar(str(c)) for c in range(len(game))] for i in range(len(game))]

# set all places where a number is to zero
for i in range(len(game)):
    for j in range(len(game)):
        if not game[i][j] == "":
            board[i][j] = 0

# sum of neighbour-cells must be equal to number in cell
for i in range(len(game)):
    for j in range(len(game)):
        if not game[i][j] == "":
            neighbour_cells = neighbours(i,j)
            model.Add(cp_model.LinearExpr.Sum([board[y][x] for (y,x) in neighbour_cells]) == int(game[i][j]))

Callback for solution printing

In [7]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    
    def __init__(self, variables, game):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        
    def on_solution_callback(self):
        for i in range(len(self.__variables)):
            for j in range(len(self.__variables)):
                if self.Value(self.__variables[i][j]) == 1:
                    print("[💣]\t", end='')
                elif game[i][j] == "":
                    print("[  ]\t", end='')
                else:
                    print("[ {}]\t".format(game[i][j]), end='')
            print("\n")
        print("\n\n")

Solve and print all solutions

In [8]:
solver = cp_model.CpSolver()

# for some strange reason, the same solution is printed over and over again
_ = solver.SearchForAllSolutions(model, SolutionPrinter(board, game))

[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	

[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	

[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[ 

...


[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4


[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]

[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	[ 3]	[  ]	

[ 1]	[  ]	[  ]	[💣]	[💣]	




[ 2]	[💣]	[💣]	[💣]	[ 1]	

[ 2]	[💣]	[ 4]	[ 3]	[  ]	

[  ]	[ 2]	[  ]	[ 1]	[💣]	

[💣]	[ 1]	[  ]	

In [9]:
print(f"Runtime:   {solver.WallTime()}ms")
print(f"Booleans:  {solver.NumBooleans()}")
print(f"Failures:  {solver.NumConflicts()}")
print(f"Branches:  {solver.NumBranches()}")

Runtime:   4.366072373000001ms
Booleans:  25
Failures:  0
Branches:  5140
